In [1]:
import ITensors:op
import Pkg; Pkg.add("HDF5")
import Pkg; Pkg.add("Plots")
import Pkg; Pkg.add("PyFormattedStrings")
import Pkg; Pkg.add("LinearAlgebra")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [2]:
using ITensors, ITensorMPS
import ITensors:op
using HDF5
using Plots
using PyFormattedStrings
using LinearAlgebra

In [3]:
include("overload_Spin=3h.jl")

In [4]:
using ITensors, ITensorMPS
ITensors.disable_warn_order()
let
A = 1
D = 5
N = 8

  # Make an array of N Index objects with alternating
  # "S=1/2" and "S=1" tags on odd versus even sites
  # (The first argument n->isodd(n) ... is an
  # on-the-fly function mapping integers to strings)
  sites = siteinds(n->isodd(n) ? "S=1/2" : "S=3/2",N)

  # Couplings between spin-half and
  # spin-one sites:
  Jho = 1 # half-one coupling
  Jhh = 1 # half-half coupling
  Joo = 1 # one-one coupling

  os = OpSum()
  for j=1:N-1
    os += 0.5*Jho,"S+",j,"S-",j+1
    os += 0.5*Jho,"S-",j,"S+",j+1
    os += Jho,"Sz",j,"Sz",j+1
  end
  for j=1:2:N-2
    os += 0.5*Jhh,"S+",j,"S-",j+2
    os += 0.5*Jhh,"S-",j,"S+",j+2
    os += Jhh,"Sz",j,"Sz",j+2
  end
  for j=2:2:N-2
    os += 0.5*Joo,"S+",j,"S-",j+2
    os += 0.5*Joo,"S-",j,"S+",j+2
    os += Joo,"Sz",j,"Sz",j+2
  end
    os += 0.5*Jho,"S+",N,"S-",1
    os += 0.5*Jho,"S-",N,"S+",1
    os += Jho,"Sz",N,"Sz",1
    
  for j=1:N
    os += -0.1, "Sz",j
  end
    
  H = MPO(os,sites)

nsweeps = 5
maxdim = [100, 200, 500, 1000, 2000, 5000, 10000, 25580]
cutoff = [1E-8]

psi0 = random_mps(sites;linkdims=2)

energy,psi1 = dmrg(H,psi0;nsweeps,maxdim,cutoff)
    
# Create the density matrix rho
a1 = 0
N1 = 0
a2 = 0
N2 = 0
a3 = 0
N3 = 0
a4 = 0
N4 = 0
a5 = 0
N5 = 0
a6 = 0
N6 = 0
a7 = 0
N7 = 0
orthogonalize!(psi1,A)
ket1 = psi1[A]
for i=A+1:D
    ket1 *= psi1[i]
end
bra1 = dag(ket1)
rho = prime(ket1,"Site")*bra1
#@show inds(rho)  #<-- this shows that the indices of rho corresponding to the site between B and C are the 3rd and 7th element of the inds(rho) vector.    
rho_ABC3 = rho * delta(inds(rho)[3],inds(rho)[8])     # delta(inds,inds) traces out the given inds (see the documentation)  
#rho_ABC3 = rho_ABC*delta(inds(rho)[4],inds(rho)[10])   
#rho_ABC3 = rho_ABC1*delta(inds(rho)[5],inds(rho)[12])  
#rho_ABC3 = rho_ABC2*delta(inds(rho)[6],inds(rho)[14]) 
#da, ua = eigen(rho_ABC1)


oldinds = inds(rho_ABC3)
newinds = [oldinds[5],oldinds[6],oldinds[3],oldinds[4],oldinds[1], oldinds[2], oldinds[7], oldinds[8]]
newinds1 = [oldinds[5],oldinds[2],oldinds[7],oldinds[4],oldinds[1], oldinds[6], oldinds[3], oldinds[8]]
newinds2 = [oldinds[5],oldinds[2],oldinds[3],oldinds[8],oldinds[1], oldinds[6], oldinds[7], oldinds[4]]
newinds3 = [oldinds[5],oldinds[2],oldinds[3],oldinds[4],oldinds[1], oldinds[6], oldinds[7], oldinds[8]]
newinds4 = [oldinds[1],oldinds[6],oldinds[3],oldinds[4],oldinds[5], oldinds[2], oldinds[7], oldinds[8]]
newinds5 = [oldinds[1],oldinds[2],oldinds[7],oldinds[4],oldinds[5], oldinds[6], oldinds[3], oldinds[8]]
newinds6 = [oldinds[1],oldinds[2],oldinds[3],oldinds[8],oldinds[5], oldinds[6], oldinds[7], oldinds[4]]


rho_pt = swapinds(rho_ABC3,oldinds,newinds)
rho_pt1 = swapinds(rho_ABC3,oldinds,newinds1)
rho_pt2 = swapinds(rho_ABC3,oldinds,newinds2)
rho_pt3 = swapinds(rho_ABC3,oldinds,newinds3)
rho_pt4 = swapinds(rho_ABC3,oldinds,newinds4)
rho_pt5 = swapinds(rho_ABC3,oldinds,newinds5)
rho_pt6 = swapinds(rho_ABC3,oldinds,newinds6)

# Compute the eigenvalues of the partially transposed density matrix
da, ua = eigen(rho_pt)
da1, ua1 = eigen(rho_pt1)
da2, ua2 = eigen(rho_pt2)
da3, ua3 = eigen(rho_pt3)
da4, u4 = eigen(rho_pt4)
da5, ua5 = eigen(rho_pt5)
da6, ua6 = eigen(rho_pt6)

# Calculate the negativity
for j = 1:length(diag(real(da)))
    a1 += diag(real(da))[j]
end


for j = 1:length(diag(real(da)))
    N1 += 0.5 * (abs(diag(real(da))[j]) - diag(real(da))[j])
end
L1 = log(1 + 2 * N1)


for j = 1:length(diag(real(da1)))
    N2 += 0.5 * (abs(diag(real(da1))[j]) - diag(real(da1))[j])
end
L2 = log(1 + 2 * N2)

for j = 1:length(diag(real(da2)))
    N3 += 0.5 * (abs(diag(real(da2))[j]) - diag(real(da2))[j])
end
L3 = log(1 + 2 * N3)


for j = 1:length(diag(real(da3)))
    N4 += 0.5 * (abs(diag(real(da3))[j]) - diag(real(da3))[j])
end
L4 = log(1 + 2 * N4)


for j = 1:length(diag(real(da4)))
    N5 += 0.5 * (abs(diag(real(da4))[j]) - diag(real(da4))[j])
end
L5 = log(1 + 2 * N5)

for j = 1:length(diag(real(da5)))
    N6 += 0.5 * (abs(diag(real(da5))[j]) - diag(real(da5))[j])
end
L6 = log(1 + 2 * N6)


for j = 1:length(diag(real(da6)))
    N7 += 0.5 * (abs(diag(real(da6))[j]) - diag(real(da6))[j])
end
L7 = log(1 + 2 * N7)

L11=(L1*L2*L3*L4*L5*L6*L7)^(1/7)
@show L11;
    
    
end   

After sweep 1 energy=-10.959288396499602  maxlinkdim=16 maxerr=3.99E-16 time=35.530
After sweep 2 energy=-11.727414000716045  maxlinkdim=60 maxerr=8.13E-09 time=0.164
After sweep 3 energy=-11.89972188990199  maxlinkdim=63 maxerr=1.17E-09 time=0.197
After sweep 4 energy=-11.905907222374244  maxlinkdim=64 maxerr=9.04E-09 time=0.122
After sweep 5 energy=-11.906180541922431  maxlinkdim=64 maxerr=9.32E-09 time=0.145
L11 = 0.47331498651742604


0.47331498651742604

In [ ]:
using ITensors, ITensorMPS
ITensors.disable_warn_order()
let
A = 1
D = 6
N = 14

  sites = siteinds(n->isodd(n) ? "S=1/2" : "S=3/2",N)

  Jho = 1
  Jhh = 1 
  Joo = 1

  os = OpSum()
  for j=1:N-1
    os += 0.5*Jho,"S+",j,"S-",j+1
    os += 0.5*Jho,"S-",j,"S+",j+1
    os += Jho,"Sz",j,"Sz",j+1
  end
  for j=1:2:N-2
    os += 0.5*Jhh,"S+",j,"S-",j+2
    os += 0.5*Jhh,"S-",j,"S+",j+2
    os += Jhh,"Sz",j,"Sz",j+2
  end
  for j=2:2:N-2
    os += 0.5*Joo,"S+",j,"S-",j+2
    os += 0.5*Joo,"S-",j,"S+",j+2
    os += Joo,"Sz",j,"Sz",j+2
  end
    os += 0.5*Jho,"S+",N,"S-",1
    os += 0.5*Jho,"S-",N,"S+",1
    os += Jho,"Sz",N,"Sz",1
    
  for j=1:N
    os += -0.1, "Sz",j
  end
    
  H = MPO(os,sites)

nsweeps = 5
maxdim = [100, 200, 500, 1000, 2000, 5000, 10000, 25580]
cutoff = [1E-8]

psi0 = random_mps(sites;linkdims=2)

energy,psi1 = dmrg(H,psi0;nsweeps,maxdim,cutoff)
    
# Create the density matrix rho
a1 = 0
N1 = 0
a2 = 0
N2 = 0
a3 = 0
N3 = 0
a4 = 0
N4 = 0
a5 = 0
N5 = 0
a6 = 0
N6 = 0
a7 = 0
N7 = 0
orthogonalize!(psi1,4)
ket1 = psi1[A]
for i=A+1:D
    ket1 *= psi1[i]
end
bra1 = dag(ket1)
rho = prime(ket1,"Site")*bra1
#@show inds(rho) 
rho_ABC = rho * delta(inds(rho)[3],inds(rho)[9])    
rho_ABC3 = rho_ABC*delta(inds(rho)[4],inds(rho)[10])   



oldinds = inds(rho_ABC3)
newinds = [oldinds[5],oldinds[6],oldinds[3],oldinds[4],oldinds[1], oldinds[2], oldinds[7], oldinds[8]]
newinds1 = [oldinds[5],oldinds[2],oldinds[7],oldinds[4],oldinds[1], oldinds[6], oldinds[3], oldinds[8]]
newinds2 = [oldinds[5],oldinds[2],oldinds[3],oldinds[8],oldinds[1], oldinds[6], oldinds[7], oldinds[4]]
newinds3 = [oldinds[5],oldinds[2],oldinds[3],oldinds[4],oldinds[1], oldinds[6], oldinds[7], oldinds[8]]
newinds4 = [oldinds[1],oldinds[6],oldinds[3],oldinds[4],oldinds[5], oldinds[2], oldinds[7], oldinds[8]]
newinds5 = [oldinds[1],oldinds[2],oldinds[7],oldinds[4],oldinds[5], oldinds[6], oldinds[3], oldinds[8]]
newinds6 = [oldinds[1],oldinds[2],oldinds[3],oldinds[8],oldinds[5], oldinds[6], oldinds[7], oldinds[4]]


rho_pt = swapinds(rho_ABC3,oldinds,newinds)
rho_pt1 = swapinds(rho_ABC3,oldinds,newinds1)
rho_pt2 = swapinds(rho_ABC3,oldinds,newinds2)
rho_pt3 = swapinds(rho_ABC3,oldinds,newinds3)
rho_pt4 = swapinds(rho_ABC3,oldinds,newinds4)
rho_pt5 = swapinds(rho_ABC3,oldinds,newinds5)
rho_pt6 = swapinds(rho_ABC3,oldinds,newinds6)


da, ua = eigen(rho_pt)
da1, ua1 = eigen(rho_pt1)
da2, ua2 = eigen(rho_pt2)
da3, ua3 = eigen(rho_pt3)
da4, u4 = eigen(rho_pt4)
da5, ua5 = eigen(rho_pt5)
da6, ua6 = eigen(rho_pt6)

# Calculate the negativity
for j = 1:length(diag(real(da)))
    a1 += diag(real(da))[j]
end


for j = 1:length(diag(real(da)))
    N1 += 0.5 * (abs(diag(real(da))[j]) - diag(real(da))[j])
end
L1 = log(1 + 2 * N1)


for j = 1:length(diag(real(da1)))
    N2 += 0.5 * (abs(diag(real(da1))[j]) - diag(real(da1))[j])
end
L2 = log(1 + 2 * N2)

for j = 1:length(diag(real(da2)))
    N3 += 0.5 * (abs(diag(real(da2))[j]) - diag(real(da2))[j])
end
L3 = log(1 + 2 * N3)


for j = 1:length(diag(real(da3)))
    N4 += 0.5 * (abs(diag(real(da3))[j]) - diag(real(da3))[j])
end
L4 = log(1 + 2 * N4)


for j = 1:length(diag(real(da4)))
    N5 += 0.5 * (abs(diag(real(da4))[j]) - diag(real(da4))[j])
end
L5 = log(1 + 2 * N5)

for j = 1:length(diag(real(da5)))
    N6 += 0.5 * (abs(diag(real(da5))[j]) - diag(real(da5))[j])
end
L6 = log(1 + 2 * N6)


for j = 1:length(diag(real(da6)))
    N7 += 0.5 * (abs(diag(real(da6))[j]) - diag(real(da6))[j])
end
L7 = log(1 + 2 * N7)

L22=(L1*L2*L3*L4*L5*L6*L7)^(1/7)
@show L22;
    
    
end   

In [ ]:
using ITensors, ITensorMPS
ITensors.disable_warn_order()
let
A = 1
D = 7
N = 14

  sites = siteinds(n->isodd(n) ? "S=1/2" : "S=3/2",N)

  Jho = 1
  Jhh = 1 
  Joo = 1 

  os = OpSum()
  for j=1:N-1
    os += 0.5*Jho,"S+",j,"S-",j+1
    os += 0.5*Jho,"S-",j,"S+",j+1
    os += Jho,"Sz",j,"Sz",j+1
  end
  for j=1:2:N-2
    os += 0.5*Jhh,"S+",j,"S-",j+2
    os += 0.5*Jhh,"S-",j,"S+",j+2
    os += Jhh,"Sz",j,"Sz",j+2
  end
  for j=2:2:N-2
    os += 0.5*Joo,"S+",j,"S-",j+2
    os += 0.5*Joo,"S-",j,"S+",j+2
    os += Joo,"Sz",j,"Sz",j+2
  end
    os += 0.5*Jho,"S+",N,"S-",1
    os += 0.5*Jho,"S-",N,"S+",1
    os += Jho,"Sz",N,"Sz",1

  for j=1:N
    os += -0.1, "Sz",j
  end
    
  H = MPO(os,sites)

nsweeps = 5
maxdim = [100, 200, 500, 1000, 2000, 5000, 10000, 25580]
cutoff = [1E-8]

psi0 = random_mps(sites;linkdims=2)

energy,psi1 = dmrg(H,psi0;nsweeps,maxdim,cutoff)
    
# Create the density matrix rho
a1 = 0
N1 = 0
a2 = 0
N2 = 0
a3 = 0
N3 = 0
a4 = 0
N4 = 0
a5 = 0
N5 = 0
a6 = 0
N6 = 0
a7 = 0
N7 = 0
orthogonalize!(psi1,4)
ket1 = psi1[A]
for i=A+1:D
    ket1 *= psi1[i]
end
bra1 = dag(ket1)
rho = prime(ket1,"Site")*bra1
#@show inds(rho) 
rho_ABC = rho * delta(inds(rho)[3],inds(rho)[10])    
rho_ABC1 = rho_ABC*delta(inds(rho)[4],inds(rho)[11])   
rho_ABC3 = rho_ABC1*delta(inds(rho)[5],inds(rho)[12]) 


oldinds = inds(rho_ABC3)
newinds = [oldinds[5],oldinds[6],oldinds[3],oldinds[4],oldinds[1], oldinds[2], oldinds[7], oldinds[8]]
newinds1 = [oldinds[5],oldinds[2],oldinds[7],oldinds[4],oldinds[1], oldinds[6], oldinds[3], oldinds[8]]
newinds2 = [oldinds[5],oldinds[2],oldinds[3],oldinds[8],oldinds[1], oldinds[6], oldinds[7], oldinds[4]]
newinds3 = [oldinds[5],oldinds[2],oldinds[3],oldinds[4],oldinds[1], oldinds[6], oldinds[7], oldinds[8]]
newinds4 = [oldinds[1],oldinds[6],oldinds[3],oldinds[4],oldinds[5], oldinds[2], oldinds[7], oldinds[8]]
newinds5 = [oldinds[1],oldinds[2],oldinds[7],oldinds[4],oldinds[5], oldinds[6], oldinds[3], oldinds[8]]
newinds6 = [oldinds[1],oldinds[2],oldinds[3],oldinds[8],oldinds[5], oldinds[6], oldinds[7], oldinds[4]]


rho_pt = swapinds(rho_ABC3,oldinds,newinds)
rho_pt1 = swapinds(rho_ABC3,oldinds,newinds1)
rho_pt2 = swapinds(rho_ABC3,oldinds,newinds2)
rho_pt3 = swapinds(rho_ABC3,oldinds,newinds3)
rho_pt4 = swapinds(rho_ABC3,oldinds,newinds4)
rho_pt5 = swapinds(rho_ABC3,oldinds,newinds5)
rho_pt6 = swapinds(rho_ABC3,oldinds,newinds6)


da, ua = eigen(rho_pt)
da1, ua1 = eigen(rho_pt1)
da2, ua2 = eigen(rho_pt2)
da3, ua3 = eigen(rho_pt3)
da4, u4 = eigen(rho_pt4)
da5, ua5 = eigen(rho_pt5)
da6, ua6 = eigen(rho_pt6)

# Calculate the negativity
for j = 1:length(diag(real(da)))
    a1 += diag(real(da))[j]
end


for j = 1:length(diag(real(da)))
    N1 += 0.5 * (abs(diag(real(da))[j]) - diag(real(da))[j])
end
L1 = log(1 + 2 * N1)


for j = 1:length(diag(real(da1)))
    N2 += 0.5 * (abs(diag(real(da1))[j]) - diag(real(da1))[j])
end
L2 = log(1 + 2 * N2)

for j = 1:length(diag(real(da2)))
    N3 += 0.5 * (abs(diag(real(da2))[j]) - diag(real(da2))[j])
end
L3 = log(1 + 2 * N3)


for j = 1:length(diag(real(da3)))
    N4 += 0.5 * (abs(diag(real(da3))[j]) - diag(real(da3))[j])
end
L4 = log(1 + 2 * N4)


for j = 1:length(diag(real(da4)))
    N5 += 0.5 * (abs(diag(real(da4))[j]) - diag(real(da4))[j])
end
L5 = log(1 + 2 * N5)

for j = 1:length(diag(real(da5)))
    N6 += 0.5 * (abs(diag(real(da5))[j]) - diag(real(da5))[j])
end
L6 = log(1 + 2 * N6)


for j = 1:length(diag(real(da6)))
    N7 += 0.5 * (abs(diag(real(da6))[j]) - diag(real(da6))[j])
end
L7 = log(1 + 2 * N7)

L33=(L1*L2*L3*L4*L5*L6*L7)^(1/7)
@show L33;
    
    
end   

In [ ]:
using ITensors, ITensorMPS
ITensors.disable_warn_order()
let
A = 1
D = 8
N = 14

  sites = siteinds(n->isodd(n) ? "S=1/2" : "S=3/2",N)

  Jho = 1 
  Jhh = 1 
  Joo = 1 

  os = OpSum()
  for j=1:N-1
    os += 0.5*Jho,"S+",j,"S-",j+1
    os += 0.5*Jho,"S-",j,"S+",j+1
    os += Jho,"Sz",j,"Sz",j+1
  end
  for j=1:2:N-2
    os += 0.5*Jhh,"S+",j,"S-",j+2
    os += 0.5*Jhh,"S-",j,"S+",j+2
    os += Jhh,"Sz",j,"Sz",j+2
  end
  for j=2:2:N-2
    os += 0.5*Joo,"S+",j,"S-",j+2
    os += 0.5*Joo,"S-",j,"S+",j+2
    os += Joo,"Sz",j,"Sz",j+2
  end
    os += 0.5*Jho,"S+",N,"S-",1
    os += 0.5*Jho,"S-",N,"S+",1
    os += Jho,"Sz",N,"Sz",1
    
  for j=1:N
    os += -0.1, "Sz",j
  end
  H = MPO(os,sites)

nsweeps = 5
maxdim = [100, 200, 500, 1000, 2000, 5000, 10000, 25580]
cutoff = [1E-8]

psi0 = random_mps(sites;linkdims=10)

energy,psi1 = dmrg(H,psi0;nsweeps,maxdim,cutoff)
    
# Create the density matrix rho
a1 = 0
N1 = 0
a2 = 0
N2 = 0
a3 = 0
N3 = 0
a4 = 0
N4 = 0
a5 = 0
N5 = 0
a6 = 0
N6 = 0
a7 = 0
N7 = 0
orthogonalize!(psi1,4)
ket1 = psi1[A]
for i=A+1:D
    ket1 *= psi1[i]
end
bra1 = dag(ket1)
rho = prime(ket1,"Site")*bra1
#@show inds(rho) 
rho_ABC = rho * delta(inds(rho)[3],inds(rho)[11])    
rho_ABC1 = rho_ABC*delta(inds(rho)[4],inds(rho)[12])   
rho_ABC2 = rho_ABC1*delta(inds(rho)[5],inds(rho)[13]) 
rho_ABC3 = rho_ABC2*delta(inds(rho)[6],inds(rho)[14])

oldinds = inds(rho_ABC3)
newinds = [oldinds[5],oldinds[6],oldinds[3],oldinds[4],oldinds[1], oldinds[2], oldinds[7], oldinds[8]]
newinds1 = [oldinds[5],oldinds[2],oldinds[7],oldinds[4],oldinds[1], oldinds[6], oldinds[3], oldinds[8]]
newinds2 = [oldinds[5],oldinds[2],oldinds[3],oldinds[8],oldinds[1], oldinds[6], oldinds[7], oldinds[4]]
newinds3 = [oldinds[5],oldinds[2],oldinds[3],oldinds[4],oldinds[1], oldinds[6], oldinds[7], oldinds[8]]
newinds4 = [oldinds[1],oldinds[6],oldinds[3],oldinds[4],oldinds[5], oldinds[2], oldinds[7], oldinds[8]]
newinds5 = [oldinds[1],oldinds[2],oldinds[7],oldinds[4],oldinds[5], oldinds[6], oldinds[3], oldinds[8]]
newinds6 = [oldinds[1],oldinds[2],oldinds[3],oldinds[8],oldinds[5], oldinds[6], oldinds[7], oldinds[4]]


rho_pt = swapinds(rho_ABC3,oldinds,newinds)
rho_pt1 = swapinds(rho_ABC3,oldinds,newinds1)
rho_pt2 = swapinds(rho_ABC3,oldinds,newinds2)
rho_pt3 = swapinds(rho_ABC3,oldinds,newinds3)
rho_pt4 = swapinds(rho_ABC3,oldinds,newinds4)
rho_pt5 = swapinds(rho_ABC3,oldinds,newinds5)
rho_pt6 = swapinds(rho_ABC3,oldinds,newinds6)


da, ua = eigen(rho_pt)
da1, ua1 = eigen(rho_pt1)
da2, ua2 = eigen(rho_pt2)
da3, ua3 = eigen(rho_pt3)
da4, u4 = eigen(rho_pt4)
da5, ua5 = eigen(rho_pt5)
da6, ua6 = eigen(rho_pt6)

# Calculate the negativity
for j = 1:length(diag(real(da)))
    a1 += diag(real(da))[j]
end


for j = 1:length(diag(real(da)))
    N1 += 0.5 * (abs(diag(real(da))[j]) - diag(real(da))[j])
end
L1 = log(1 + 2 * N1)


for j = 1:length(diag(real(da1)))
    N2 += 0.5 * (abs(diag(real(da1))[j]) - diag(real(da1))[j])
end
L2 = log(1 + 2 * N2)

for j = 1:length(diag(real(da2)))
    N3 += 0.5 * (abs(diag(real(da2))[j]) - diag(real(da2))[j])
end
L3 = log(1 + 2 * N3)


for j = 1:length(diag(real(da3)))
    N4 += 0.5 * (abs(diag(real(da3))[j]) - diag(real(da3))[j])
end
L4 = log(1 + 2 * N4)


for j = 1:length(diag(real(da4)))
    N5 += 0.5 * (abs(diag(real(da4))[j]) - diag(real(da4))[j])
end
L5 = log(1 + 2 * N5)

for j = 1:length(diag(real(da5)))
    N6 += 0.5 * (abs(diag(real(da5))[j]) - diag(real(da5))[j])
end
L6 = log(1 + 2 * N6)


for j = 1:length(diag(real(da6)))
    N7 += 0.5 * (abs(diag(real(da6))[j]) - diag(real(da6))[j])
end
L7 = log(1 + 2 * N7)

L44=(L1*L2*L3*L4*L5*L6*L7)^(1/7)
@show L44;
    
    
end   